In [ ]:
# Application streamlit code, see details above
# import os, pathlib, shutil, random
# from pathlib import Path
# import tensorflow as tf
# from tensorflow.keras import layers
# from tensorflow import keras
# from tensorflow.keras.layers import TextVectorization
# import streamlit as st

# st.title('Movie sentiment across models')

# #Global variables
# maxTokens = 20000 # max vocabulary size
# max_length=600 # max document size
# batch_size=32
# nlpRoot="C:/Users/plermant/git/miniprojects/NLPpractice"

# trainPath=nlpRoot+"/aclImdb/train"

# train_ds = keras.utils.text_dataset_from_directory(trainPath, batch_size=batch_size)
# text_only_train_ds = train_ds.map(lambda x, y: x)

# text_vectorizationTfIdf = TextVectorization(ngrams=2,max_tokens=maxTokens,output_mode="tf-idf")	
# text_vectorizationTfIdf.adapt(text_only_train_ds)

# text_vectorizationBidiLSTM = layers.TextVectorization(max_tokens=maxTokens,output_mode="int",output_sequence_length=max_length,)
# text_vectorizationBidiLSTM.adapt(text_only_train_ds)

# def modelFromName(name):
#     path=nlpRoot+"/models/"+name
#     if name=='bagBigramTdIdf' or name =='embedBidiLSTM':
#         return tf.keras.models.load_model(path)	
#     else:
#         print("Error in model name, does not exist, aborting ...")
#         exit(1)
        
# def predictSentiment(inString,modelName):
#     model=modelFromName(modelName)
#     inputs = keras.Input(shape=(1,), dtype="string")
#     if modelName=='bagBigramTdIdf':
#         processed_inputs = text_vectorizationTfIdf(inputs)
#     elif modelName=='embedBidiLSTM':
#         processed_inputs = text_vectorizationBidiLSTM(inputs)
#     outputs = model(processed_inputs)
#     inference_model = keras.Model(inputs, outputs)
#     predictions = inference_model(inString) 
#     resp=round(float(predictions[0] * 100),2)
#     return resp
	
# modelName= st.selectbox("Select a ML model from this list",("bagBigramTdIdf", 'embedBidiLSTM',"more to come ..."))
# inputString= st.text_input('What do you think of the movie? e.g. "I loved the movie", or "This movie sucked"')

# sentimentPercent=predictSentiment(tf.convert_to_tensor([[inputString],]),modelName)
# st.write("Sentiment value is",sentimentPercent,"per",modelName,"model.")

In [13]:
import os, pathlib, shutil, random
from pathlib import Path
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization

In [14]:
#Global variables
percentVal=.2
batch_size = 32
maxTokens = 20000 # max voclbulary size
max_length = 600 # max number of words in document
epochCount=2
nlpRoot="C:/Users/plermant/git/miniprojects/NLPpractice"

In [15]:
#fetch files and untar, to be performed only once in project
#!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
#!tar -xf aclImdb_v1.tar.gz

In [16]:
#Put percentVal of samples in a validation directory, to be performed only once in project
# base_dir = pathlib.Path("aclImdb")
# val_dir = base_dir / "val" 
# train_dir = base_dir / "train" 
# for category in ("neg", "pos"):
#     os.makedirs(val_dir / category)
#     files = os.listdir(train_dir / category)
#     random.Random(1337).shuffle(files)
#     num_val_samples = int(percentVal * len(files))
#     val_files = files[-num_val_samples:]
#     for fname in val_files:
#         shutil.move(train_dir / category / fname,
#                     val_dir / category / fname)

In [17]:
# Let’s create three Dataset objects for training, validation, and testing:
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [18]:
# Displaying the shapes and dtypes of the first batch
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break


inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'It never ceases to amaze me how you can take an excellent actor, and put him to waste in a film such as this. Robert De Niro is one of the best Hollywood stars of all time, but even he couldn\'t save this movie. In fact, his character is much the same as the one he played in Cape Fear, which was actually pretty good, but I can\'t stand it when actors do the same schtick over more than one movie. I believe it gets old, and that is the case here.<br /><br />There\'s nothing surprising in this movie, but then, the story has been told a million times before. Wesley Snipes is your typical baseball player, and his conceit shows through in his characterization. De Niro plays the obsessed fan, but his role in this film is less than entertaining.<br /><br />However, because De Niro is IN this film, that makes it a draw if you are a fan (no pun intended) who sees everyt

In [21]:
# Our model-building utility
  
def get_model(max_tokens=maxTokens, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

In [25]:
# Configuring the TextVectorization layer to return bigrams with TF-IDF
text_vectorizationTfIdf = TextVectorization(
    ngrams=2,
    max_tokens=maxTokens,
    output_mode="tf-idf" 
)

In [26]:
# Training and testing the TF-IDF bigram model
text_vectorizationTfIdf.adapt(text_only_train_ds)
 
tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorizationTfIdf(x), y),
    num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorizationTfIdf(x), y),
    num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorizationTfIdf(x), y),
    num_parallel_calls=4)
 
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",
                                    save_best_only=True)
]
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=epochCount,
          callbacks=callbacks)
model = keras.models.load_model("tfidf_2gram.keras")
print(f"tfidf_2gram test accuracy: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
625/625 [==============================] - 18s 28ms/step - loss: 0.5199 - accuracy: 0.7596 - val_loss: 0.3482 - val_accuracy: 0.8454
Epoch 2/2
782/782 [=========================

In [27]:
# Saving/loading model
path=nlpRoot+'/models'+'/bagBigramTdIdf'
model.save(path)
model=keras.models.load_model(path)


INFO:tensorflow:Assets written to: C:/Users/plermant/git/miniprojects/NLPpractice/models/bagBigramTdIdf\assets


In [28]:
def modelFromName(name):
    path=nlpRoot+"/models/"+name
    if name=='bagBigramTdIdf'or name=='embedBidiLSTM':
        return tf.keras.models.load_model(path) 
    else:
        print("Error in model name, does not exist, aborting ...")
        exit(1)
        

In [30]:
def predictSentiment(inString,modelName):
    model=modelFromName(modelName)
    inputs = keras.Input(shape=(1,), dtype="string")
    if modelName=='bagBigramTdIdf':
        processed_inputs = text_vectorizationTfIdf(inputs)
    elif modelName=='embedBidiLSTM':
        processed_inputs = text_vectorizationBidiLSTM(inputs)
    else:
        print("Can't vectorize this model, exiting ..")
        exit(1)
    outputs = model(processed_inputs)
    inference_model = keras.Model(inputs, outputs)

    predictions = inference_model(raw_text_data) 
    resp=round(float(predictions[0] * 100),2)
    return resp
    

In [32]:
# Now lets do embedding with bidi LSTM
  
text_vectorizationBidiLSTM = layers.TextVectorization(
    max_tokens=maxTokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorizationBidiLSTM.adapt(text_only_train_ds)
 
int_train_ds = train_ds.map(
    lambda x, y: (text_vectorizationBidiLSTM(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorizationBidiLSTM(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorizationBidiLSTM(x), y),
    num_parallel_calls=4)


inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=maxTokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=epochCount,
          callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras") 
print(f"bidiLSTM test accuracy: {model.evaluate(int_test_ds)[1]:.3f}")

#Saving/loading model
path=nlpRoot+'/models'+'/embedBidiLSTM'
model.save(path)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               73984     
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

INFO:tensorflow:Assets written to: C:/Users/plermant/git/miniprojects/NLPpractice/models/embedBidiLSTM\assets


INFO:tensorflow:Assets written to: C:/Users/plermant/git/miniprojects/NLPpractice/models/embedBidiLSTM\assets


In [33]:
raw_text_data = tf.convert_to_tensor([["That was an excellent movie, I loved it."],])
#raw_text_data = tf.convert_to_tensor([["That was a horrible movie, I hated it."],])

print(predictSentiment(raw_text_data,'bagBigramTdIdf'),"percent positive for bagBigramTdIdf")
print(predictSentiment(raw_text_data,'embedBidiLSTM'),"percent positive for embedBidiLSTM")

89.27 percent positive
91.58 percent positive
